# Tokenization Exploration

Tokenization is how LLMs break text into pieces they can understand. Instead of reading character-by-character or word-by-word, they use **tokens** - chunks that can be whole words, parts of words, or even single characters.

This notebook lets you experiment hands-on with tokenization using `tiktoken`, the tokenizer OpenAI uses for GPT models.

## 1. Setup

First, let's install and import the tokenizer.

In [1]:
# Install tiktoken if you don't have it
!pip install tiktoken -q

In [2]:
import tiktoken

# Load the encoding used by GPT-4 and GPT-3.5-turbo
enc = tiktoken.get_encoding("cl100k_base")

print("Tokenizer loaded!")
print(f"Vocabulary size: {enc.n_vocab:,} tokens")

Tokenizer loaded!
Vocabulary size: 100,277 tokens


## 2. Basic Tokenization

Let's see how text gets converted to tokens and back.

In [3]:
# Encode text to tokens
text = "Hello world "
tokens = enc.encode(text)

print(f"Text: '{text}'")
print(f"Token IDs: {tokens}")
print(f"Number of tokens: {len(tokens)}")

Text: 'Hello world '
Token IDs: [9906, 1917, 220]
Number of tokens: 3


In [4]:
# Decode tokens back to text
decoded = enc.decode(tokens)
print(f"Decoded: '{decoded}'")
print(f"Round-trip successful: {text == decoded}")

Decoded: 'Hello world '
Round-trip successful: True


In [5]:
# See what each token represents
def show_tokens(text):
    """Display each token ID and what text it represents."""
    tokens = enc.encode(text)
    print(f"Text: '{text}'")
    print(f"Total tokens: {len(tokens)}")
    print("\nBreakdown:")
    for i, token_id in enumerate(tokens):
        token_text = enc.decode([token_id])
        print(f"  {i+1}. ID {token_id:>6} -> '{token_text}'")

show_tokens("Hello world")

Text: 'Hello world'
Total tokens: 2

Breakdown:
  1. ID   9906 -> 'Hello'
  2. ID   1917 -> ' world'


**Try it yourself:** Change the text above to see how different phrases get tokenized.

## 3. Case Sensitivity

Tokenizers treat uppercase and lowercase differently. The same word in different cases becomes different tokens.

In [6]:
# Compare different cases of the same word
words = ["ChatGPT", "chatgpt", "CHATGPT", "Chatgpt"]

for word in words:
    tokens = enc.encode(word)
    print(f"'{word}' -> {len(tokens)} tokens: {tokens}")

'ChatGPT' -> 3 tokens: [16047, 38, 2898]
'chatgpt' -> 3 tokens: [9884, 70, 418]
'CHATGPT' -> 3 tokens: [61817, 38, 2898]
'Chatgpt' -> 3 tokens: [16047, 70, 418]


In [7]:
# Look at each version in detail
print("=" * 50)
for word in ["ChatGPT", "chatgpt", "CHATGPT"]:
    show_tokens(word)
    print("=" * 50)

Text: 'ChatGPT'
Total tokens: 3

Breakdown:
  1. ID  16047 -> 'Chat'
  2. ID     38 -> 'G'
  3. ID   2898 -> 'PT'
Text: 'chatgpt'
Total tokens: 3

Breakdown:
  1. ID   9884 -> 'chat'
  2. ID     70 -> 'g'
  3. ID    418 -> 'pt'
Text: 'CHATGPT'
Total tokens: 3

Breakdown:
  1. ID  61817 -> 'CHAT'
  2. ID     38 -> 'G'
  3. ID   2898 -> 'PT'


**Insight:** "ChatGPT" (mixed case) might tokenize differently than "chatgpt" (lowercase). This means:
- Different token counts
- Different costs
- Potentially different model behavior

## 4. Word Splitting

Common words often get a single token. Rare or complex words get split into pieces.

In [8]:
# Common words - usually single tokens
common_words = ["the", "hello", "world", "is", "a", "computer"]

print("Common words:")
for word in common_words:
    tokens = enc.encode(word)
    print(f"  '{word}' -> {len(tokens)} token(s)")

Common words:
  'the' -> 1 token(s)
  'hello' -> 1 token(s)
  'world' -> 1 token(s)
  'is' -> 1 token(s)
  'a' -> 1 token(s)
  'computer' -> 1 token(s)


In [9]:
# Longer/rarer words - often split
complex_words = ["tokenization", "unbelievable", "cryptocurrency", "anthropomorphic"]

print("Complex words:")
for word in complex_words:
    show_tokens(word)
    print()

Complex words:
Text: 'tokenization'
Total tokens: 2

Breakdown:
  1. ID   5963 -> 'token'
  2. ID   2065 -> 'ization'

Text: 'unbelievable'
Total tokens: 3

Breakdown:
  1. ID    359 -> 'un'
  2. ID  32898 -> 'belie'
  3. ID  24694 -> 'vable'

Text: 'cryptocurrency'
Total tokens: 2

Breakdown:
  1. ID  49225 -> 'crypt'
  2. ID  78580 -> 'ocurrency'

Text: 'anthropomorphic'
Total tokens: 3

Breakdown:
  1. ID  32329 -> 'anth'
  2. ID    897 -> 'rop'
  3. ID  71017 -> 'omorphic'



In [10]:
# Made-up words get split heavily
made_up = ["flurbtastic", "snorkelwumpus", "blorpification", "xyzzyplugh"]

print("Made-up words:")
for word in made_up:
    show_tokens(word)
    print()

Made-up words:
Text: 'flurbtastic'
Total tokens: 4

Breakdown:
  1. ID   1517 -> 'fl'
  2. ID   9225 -> 'urb'
  3. ID     83 -> 't'
  4. ID   5174 -> 'astic'

Text: 'snorkelwumpus'
Total tokens: 6

Breakdown:
  1. ID   9810 -> 'sn'
  2. ID    672 -> 'ork'
  3. ID    301 -> 'el'
  4. ID     86 -> 'w'
  5. ID   1538 -> 'ump'
  6. ID    355 -> 'us'

Text: 'blorpification'
Total tokens: 3

Breakdown:
  1. ID   2067 -> 'bl'
  2. ID  31215 -> 'orp'
  3. ID   2461 -> 'ification'

Text: 'xyzzyplugh'
Total tokens: 4

Breakdown:
  1. ID  29954 -> 'xyz'
  2. ID   4341 -> 'zy'
  3. ID    501 -> 'pl'
  4. ID   7595 -> 'ugh'



**Key insight:** The tokenizer learned from real text. Words it saw often become single tokens. Rare combinations get broken into smaller pieces it does recognize.

## 5. Spaces Matter

Spaces aren't always separate tokens. They often attach to the word that follows.

In [11]:
# Compare with and without leading space
print("Without leading space:")
show_tokens("Hello")
print()
print("With leading space:")
show_tokens(" Hello")

Without leading space:
Text: 'Hello'
Total tokens: 1

Breakdown:
  1. ID   9906 -> 'Hello'

With leading space:
Text: ' Hello'
Total tokens: 1

Breakdown:
  1. ID  22691 -> ' Hello'


In [12]:
# See how spaces work in sentences
show_tokens("Hello world how are you")

Text: 'Hello world how are you'
Total tokens: 5

Breakdown:
  1. ID   9906 -> 'Hello'
  2. ID   1917 -> ' world'
  3. ID   1268 -> ' how'
  4. ID    527 -> ' are'
  5. ID    499 -> ' you'


**Notice:** In the sentence above, most words after the first have their space attached. The tokenizer represents " world" (space+word) as a single token, different from "world" (no space).

This is why `" Hello"` and `"Hello"` produce different tokens!

In [13]:
# What about multiple spaces?
show_tokens("Hello  world")  # two spaces

Text: 'Hello  world'
Total tokens: 3

Breakdown:
  1. ID   9906 -> 'Hello'
  2. ID    220 -> ' '
  3. ID   1917 -> ' world'


## 6. Token Counting

Let's build a simple function to count tokens, then compare different ways of saying the same thing.

In [14]:
def count_tokens(text):
    """Count the number of tokens in a text string."""
    return len(enc.encode(text))

# Test it
print(f"'Hello world' = {count_tokens('Hello world')} tokens")

'Hello world' = 2 tokens


In [15]:
# Compare different phrasings of the same idea
phrasings = [
    "Please summarize this document.",
    "Summarize this.",
    "TL;DR",
    "Can you please provide a brief summary of the contents of this document for me?",
]

print("Same request, different token counts:\n")
for phrase in phrasings:
    tokens = count_tokens(phrase)
    print(f"{tokens:2} tokens: '{phrase}'")

Same request, different token counts:

 5 tokens: 'Please summarize this document.'
 5 tokens: 'Summarize this.'
 3 tokens: 'TL;DR'
16 tokens: 'Can you please provide a brief summary of the contents of this document for me?'


In [16]:
# Numbers can be tricky
numbers = ["1", "12", "123", "1234", "12345", "123456789"]

print("Number tokenization:")
for num in numbers:
    show_tokens(num)
    print()

Number tokenization:
Text: '1'
Total tokens: 1

Breakdown:
  1. ID     16 -> '1'

Text: '12'
Total tokens: 1

Breakdown:
  1. ID    717 -> '12'

Text: '123'
Total tokens: 1

Breakdown:
  1. ID   4513 -> '123'

Text: '1234'
Total tokens: 2

Breakdown:
  1. ID   4513 -> '123'
  2. ID     19 -> '4'

Text: '12345'
Total tokens: 2

Breakdown:
  1. ID   4513 -> '123'
  2. ID   1774 -> '45'

Text: '123456789'
Total tokens: 3

Breakdown:
  1. ID   4513 -> '123'
  2. ID  10961 -> '456'
  3. ID  16474 -> '789'



**Experiment:** Try writing the same prompt two different ways and compare the token counts.

In [17]:
# Your turn: compare two prompts
prompt1 = "Explain quantum computing to me like I'm five years old."
prompt2 = "Explain quantum computing simply."

print(f"Prompt 1: {count_tokens(prompt1)} tokens")
print(f"Prompt 2: {count_tokens(prompt2)} tokens")
print(f"Difference: {count_tokens(prompt1) - count_tokens(prompt2)} tokens")

Prompt 1: 13 tokens
Prompt 2: 6 tokens
Difference: 7 tokens


## 7. Practical Implications

Understanding tokenization matters for two big reasons: **cost** and **context limits**.

### API Costs

OpenAI and other providers charge per token. Both input and output tokens cost money.

In [18]:
# Rough cost calculation (prices change - these are illustrative)
# GPT-4 Turbo pricing as of early 2024: $0.01/1K input, $0.03/1K output

def estimate_cost(text, input_price_per_1k=0.01, output_price_per_1k=0.03):
    """Estimate cost for a prompt (assuming similar length response)."""
    tokens = count_tokens(text)
    input_cost = (tokens / 1000) * input_price_per_1k
    # Assume output is roughly same length (very rough estimate)
    output_cost = (tokens / 1000) * output_price_per_1k
    return tokens, input_cost, output_cost

# Compare verbose vs concise
verbose = """I would greatly appreciate it if you could please provide me with 
a comprehensive and detailed explanation of how machine learning algorithms 
work, including all the mathematical foundations and practical applications."""

concise = "Explain how machine learning works."

v_tokens, v_in, v_out = estimate_cost(verbose)
c_tokens, c_in, c_out = estimate_cost(concise)

print("Verbose prompt:")
print(f"  {v_tokens} tokens, ~${v_in:.4f} input cost")
print()
print("Concise prompt:")
print(f"  {c_tokens} tokens, ~${c_in:.4f} input cost")
print()
print(f"Savings per request: ~${v_in - c_in:.4f}")
print(f"At 1000 requests/day: ~${(v_in - c_in) * 1000:.2f}/day")

Verbose prompt:
  35 tokens, ~$0.0004 input cost

Concise prompt:
  7 tokens, ~$0.0001 input cost

Savings per request: ~$0.0003
At 1000 requests/day: ~$0.28/day


### Context Window Limits

Every model has a maximum context window (total tokens for input + output combined).

In [19]:
# Common context window sizes
context_windows = {
    "GPT-3.5 Turbo": 16_385,
    "GPT-4 Turbo": 128_000,
    "GPT-4o": 128_000,
    "Claude 3 Sonnet": 200_000,
    "Claude 3 Opus": 200_000,
}

# How much text fits?
sample_text = """This is a sample paragraph of text that represents typical 
content you might send to an LLM. It contains normal English sentences with 
common words and punctuation. We can use this to estimate how much content 
fits in different context windows."""

tokens_per_char = count_tokens(sample_text) / len(sample_text)

print("Approximate capacity (leaving room for response):\n")
for model, window in context_windows.items():
    # Assume we want to leave 25% for the response
    usable = int(window * 0.75)
    approx_chars = int(usable / tokens_per_char)
    approx_pages = approx_chars / 3000  # ~3000 chars per page
    print(f"{model}:")
    print(f"  {usable:,} tokens for input (~{approx_pages:.0f} pages of text)")

Approximate capacity (leaving room for response):

GPT-3.5 Turbo:
  12,288 tokens for input (~21 pages of text)
GPT-4 Turbo:
  96,000 tokens for input (~165 pages of text)
GPT-4o:
  96,000 tokens for input (~165 pages of text)
Claude 3 Sonnet:
  150,000 tokens for input (~257 pages of text)
Claude 3 Opus:
  150,000 tokens for input (~257 pages of text)


In [20]:
# Check if your text fits in context
def will_it_fit(text, model_context=128000, reserve_for_response=0.25):
    """Check if text fits in a model's context window."""
    tokens = count_tokens(text)
    available = int(model_context * (1 - reserve_for_response))
    fits = tokens <= available
    
    print(f"Text tokens: {tokens:,}")
    print(f"Available: {available:,} (reserving {reserve_for_response:.0%} for response)")
    print(f"Fits: {'Yes' if fits else 'No'}")
    if not fits:
        print(f"Over by: {tokens - available:,} tokens")
    return fits

# Test with a longer text
long_text = "The quick brown fox jumps over the lazy dog. " * 100
will_it_fit(long_text)

Text tokens: 1,001
Available: 96,000 (reserving 25% for response)
Fits: Yes


True

## Summary

Key takeaways:

1. **Tokens are not words** - They're chunks that can be whole words, word pieces, or characters
2. **Case matters** - "Hello" and "hello" are different tokens
3. **Common = compact** - Frequent words use fewer tokens than rare ones
4. **Spaces attach** - " word" (with space) is often one token, different from "word"
5. **Costs add up** - Being concise saves money at scale
6. **Context has limits** - Know your model's window size

## Playground

Use this cell to experiment with your own text:

In [21]:
# Try your own text here!
my_text = "Your text here"

show_tokens(my_text)

Text: 'Your text here'
Total tokens: 3

Breakdown:
  1. ID   7927 -> 'Your'
  2. ID   1495 -> ' text'
  3. ID   1618 -> ' here'
